In [25]:
import os
import time
import gym
import torch
from pong.script_pong_new_memory.DQN import DQN # Get Network
from itertools import count
import numpy as np
import torchvision.transforms as T
from pong.script_pong_new_memory.GetScreen import get_screen
from PIL import Image

from base64 import b64encode
import matplotlib.pyplot as plt
from IPython.display import HTML

In [47]:
env = gym.make('PongDeterministic-v4').unwrapped
device = "cpu"
model = DQN(4,3).to(device)
model.load_state_dict(torch.load("../model/Pong_POLICY_8.pt", map_location=torch.device('cpu')))
model.eval()

DQN(
  (conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)

In [48]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(80, interpolation=Image.CUBIC),
                    T.ToTensor()])

In [49]:
total_reward = 0
actions = np.zeros((3))

state = torch.cat((get_screen(env, resize),
                       get_screen(env, resize),
                       get_screen(env, resize),
                       get_screen(env, resize)), dim=1, out=None)

next_state = state.clone()
    
for t in count():
    # Select and perform an action
    action = model(state.to(device)).max(1)[1].view(1, 1)
    actions[action.item() - 1] += 1
    _, reward, done, _ = env.step(action.item())
    total_reward += reward
    reward = torch.tensor([reward], device=device)
    
    # Observe new state
    current_screen = get_screen(env, resize)
    if not done:
        next_state[:, 1:4, :, :] = next_state[:, 0:3, :, :].clone()
        next_state[:, 0, :, :] = current_screen
        # next_state = current_screen - last_screen
    else:
        next_state = None
    # Move to the next state
    state = next_state
    # Perform one step of the optimization (on the target network)
    if done:
        print("Total reward: ", total_reward, "Actions: ", actions)
        # plot_durations()
        break

RuntimeError: shape '[-1, 3136]' is invalid for input of size 2304

In [50]:
def update_state(state, done=False):
      current_screen = get_screen()
      next_state_update = torch.zeros(state.shape)

      if not done:
          next_state_update[:,1:4,:,:] = state[:,0:3,:,:]
          next_state_update[:,0,:,:] = current_screen 
      else:
          next_state_update = None

      return next_state_update

In [51]:
def test(env, n_episodes, policy, path = "", render=True):
    env = gym.wrappers.Monitor(env, path, force=True)
    state = torch.cat((get_screen(env, resize),get_screen(env, resize),get_screen(env, resize),get_screen(env, resize)), dim=1, out=None)
    for episode in range(n_episodes):
        obs = env.reset()
        total_reward = 0.0
        for t in count():
            action = policy(state.to('cpu')).max(1)[1].view(1,1)

            if render:
                env.render()
                time.sleep(0.02)

            obs, reward, done, info = env.step(action)

            total_reward += reward

            state = update_state(state, done)

            if done:
                print("Finished Episode {} with reward {}".format(episode, total_reward))
                break

    env.close()
    return

In [52]:
def find_files(root, extension):
    for file in os.listdir(root):
        if file.endswith(extension):
          return file

    return None

In [53]:
path = "pong_video"
test(env, 1, model, path, render=False)
file = find_files(path, 'mp4')
file_path = path + '/' + file
print(file_path)

mp4 = open(file_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


RuntimeError: shape '[-1, 3136]' is invalid for input of size 2304